# 📌 Objectif  : Régression (Supervisé) :
Employabilité d’un candidat : Prédire le délai d’employabilité (en jours) après le diplôme




## Connexion pour ssms

In [1]:
import pyodbc
import pandas as pd

# Paramètres de connexion
server = 'LAPTOP-VAILS20L' 
database = 'DW_PI' 
username = 'sa'  
password = 'sa123' 
driver = '{SQL Server}' 

# Connexion à SQL Server
try:
    conn = pyodbc.connect(
        f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'
    )
    print("Connexion réussie !")
except Exception as e:
    print("Erreur de connexion :", e)
# Requête SQL pour joindre les tables Dim_Etudiant, Dim_Entreprise, Dim_Offre et Fact_employ1
query = """
SELECT 
    e.Etudiant_Pk, 
    e.diplome, 
    e.dateDiplome, 
    e.date_firstjob, 
    f.Fk_Entreprise, 
    f.Fk_Offre, 
    f.FK_date1, 
    f.FK_date2,
    en.name AS entreprise_name,
    o.offre AS offre_name
FROM 
    [DW_PI].[dbo].[Dim_Etudiant] e
JOIN 
    [DW_PI].[dbo].[Fact_employ1] f
ON 
    e.Etudiant_Pk = f.Etudiant_Fk
JOIN 
    [DW_PI].[dbo].[Dim_Entreprise] en
ON 
    f.Fk_Entreprise = en.Pk_Entreprise
JOIN 
    [DW_PI].[dbo].[Dim_Offre] o
ON 
    f.Fk_Offre = o.Pk_Offre
"""
# Charger les données dans un DataFrame
df = pd.read_sql(query, conn)

# Fermer la connexion
conn.close()

# Afficher les premières lignes des données pour vérifier
print(df.head())


Connexion réussie !
   Etudiant_Pk                                            diplome dateDiplome  \
0          280  Ecole Supérieure Privée d'Ingénierie et de Tec...  2013-07-13   
1          280  Ecole Supérieure Privée d'Ingénierie et de Tec...  2013-07-13   
2          280  Ecole Supérieure Privée d'Ingénierie et de Tec...  2013-07-13   
3          280  Ecole Supérieure Privée d'Ingénierie et de Tec...  2013-07-13   
4          387  Université libre de Tunis, Software engineerin...  2013-11-06   

  date_firstjob  Fk_Entreprise  Fk_Offre  FK_date1  FK_date2  \
0    2013-09-29              1        72  20130713  20130929   
1    2013-09-29              1       167  20130713  20130929   
2    2013-09-29             16        59  20130713  20130929   
3    2013-09-29             22       223  20130713  20130929   
4    2015-09-09              1       387  20131106  20150909   

    entreprise_name                             offre_name  
0  Sofrecom Tunisie                   Ingénieur

C:\Users\souha\AppData\Local\Temp\ipykernel_16848\4154534791.py:48: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [2]:
print(df.columns)


Index(['Etudiant_Pk', 'diplome', 'dateDiplome', 'date_firstjob',
       'Fk_Entreprise', 'Fk_Offre', 'FK_date1', 'FK_date2', 'entreprise_name',
       'offre_name'],
      dtype='object')


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Création de la cible : nombre de jours entre diplôme et premier emploi
df['dateDiplome'] = pd.to_datetime(df['dateDiplome'])
df['date_firstjob'] = pd.to_datetime(df['date_firstjob'])
df['jours_attente'] = (df['date_firstjob'] - df['dateDiplome']).dt.days

# Suppression des lignes avec valeurs manquantes dans la cible
df = df.dropna(subset=['jours_attente'])

# Sélection des features
features = df[['entreprise_name', 'offre_name', 'dateDiplome']]
features['annee_diplome'] = features['dateDiplome'].dt.year
features['mois_diplome'] = features['dateDiplome'].dt.month
X = features[['entreprise_name', 'offre_name', 'annee_diplome', 'mois_diplome']]
y = df['jours_attente']

# Encodage des variables catégorielles
X = pd.get_dummies(X, columns=['entreprise_name', 'offre_name'], drop_first=True)

# Split Train/Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modèles
lr = LinearRegression()
rf = RandomForestRegressor(random_state=42)

lr.fit(X_train, y_train)
rf.fit(X_train, y_train)


C:\Users\souha\AppData\Local\Temp\ipykernel_16848\1804541414.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['annee_diplome'] = features['dateDiplome'].dt.year
C:\Users\souha\AppData\Local\Temp\ipykernel_16848\1804541414.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['mois_diplome'] = features['dateDiplome'].dt.month


RandomForestRegressor(random_state=42)

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Imaginons que df soit ton DataFrame déjà préparé

# Création de la cible : nombre de jours entre diplôme et premier emploi
df['dateDiplome'] = pd.to_datetime(df['dateDiplome'])
df['date_firstjob'] = pd.to_datetime(df['date_firstjob'])
df['jours_attente'] = (df['date_firstjob'] - df['dateDiplome']).dt.days

# Suppression des lignes avec valeurs manquantes dans la cible
df = df.dropna(subset=['jours_attente'])

# Sélection des features
features = df[['entreprise_name', 'offre_name', 'dateDiplome']]
features['annee_diplome'] = features['dateDiplome'].dt.year
features['mois_diplome'] = features['dateDiplome'].dt.month
X = features[['entreprise_name', 'offre_name', 'annee_diplome', 'mois_diplome']]
y = df['jours_attente']

# Encodage des variables catégorielles
X = pd.get_dummies(X, columns=['entreprise_name', 'offre_name'], drop_first=True)

# Split Train/Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialisation du modèle RandomForestRegressor
rf = RandomForestRegressor(random_state=42)

# Entraînement du modèle
rf.fit(X_train, y_train)

# Prédictions
y_pred = rf.predict(X_test)

# Évaluation du modèle
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"MSE : {mse:.2f}")
print(f"RMSE : {rmse:.2f}")
print(f"R² : {r2:.2f}")


C:\Users\souha\AppData\Local\Temp\ipykernel_16848\661835527.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['annee_diplome'] = features['dateDiplome'].dt.year
C:\Users\souha\AppData\Local\Temp\ipykernel_16848\661835527.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['mois_diplome'] = features['dateDiplome'].dt.month


MSE : 78648.72
RMSE : 280.44
R² : 0.36


In [6]:
# Créer un nouvel échantillon de données pour la prédiction (exemple)
new_data = pd.DataFrame({
    'entreprise_name': ['Entreprise A'],  # Nom de l'entreprise
    'offre_name': ['Offre B'],  # Nom de l'offre
    'dateDiplome': [pd.to_datetime('2024-06-15')]  # Date du diplôme
})

# Ajouter les nouvelles features
new_data['annee_diplome'] = new_data['dateDiplome'].dt.year
new_data['mois_diplome'] = new_data['dateDiplome'].dt.month

# Encoder les variables catégorielles (comme pour les données d'entraînement)
new_data_encoded = pd.get_dummies(new_data[['entreprise_name', 'offre_name', 'annee_diplome', 'mois_diplome']], drop_first=True)

# Assure-toi que les colonnes d'X_train sont les mêmes que celles de new_data_encoded
# On peut faire ça en réindexant (si nécessaire)
new_data_encoded = new_data_encoded.reindex(columns=X.columns, fill_value=0)

# Faire la prédiction avec le modèle Random Forest
predicted_jours = rf.predict(new_data_encoded)

# Afficher la prédiction (le nombre de jours entre le diplôme et le premier emploi)
print(f"Le nombre estimé de jours entre le diplôme et l'emploi est : {predicted_jours[0]:.2f} jours")


Le nombre estimé de jours entre le diplôme et l'emploi est : 882.30 jours
